<a href="https://colab.research.google.com/github/sai-darshan-k/Quantum-Computing/blob/main/Sentiment_QNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pennylane pennylane-qiskit pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.8/340.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 

In [3]:
pip install pennylane

In [4]:
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import re

In [5]:
# Replace 'your_dataset.csv' with the path to your tweets dataset
df = pd.read_csv("/content/Tweets.csv")

# Example column names: 'tweet', 'sentiment'
print(df.head())




             tweet_id airline_sentiment  airline_sentiment_confidence  \
0  570306133677760513           neutral                        1.0000   
1  570301130888122368          positive                        0.3486   
2  570301083672813571           neutral                        0.6837   
3  570301031407624196          negative                        1.0000   
4  570300817074462722          negative                        1.0000   

  negativereason  negativereason_confidence         airline  \
0            NaN                        NaN  Virgin America   
1            NaN                     0.0000  Virgin America   
2            NaN                        NaN  Virgin America   
3     Bad Flight                     0.7033  Virgin America   
4     Can't Tell                     1.0000  Virgin America   

  airline_sentiment_gold        name negativereason_gold  retweet_count  \
0                    NaN     cairdin                 NaN              0   
1                    NaN    jnar

In [6]:
tweet_df = df[['text','airline_sentiment']]
print(tweet_df.shape)
tweet_df.head(5)

(14640, 2)


,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [7]:
# Select relevant columns
tweet_df = df[['text', 'airline_sentiment']]

# Check the shape and first few rows
print(tweet_df.shape)
print(tweet_df.head())

# Preprocess text (clean the 'text' column)
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text.strip()

tweet_df['cleaned_text'] = tweet_df['text'].apply(preprocess_text)

# Map 'airline_sentiment' to binary values: positive (1) and negative (0)
# You can extend this to neutral if needed
tweet_df['sentiment_label'] = tweet_df['airline_sentiment'].map({
    'positive': 1,
    'negative': 0
})


(14640, 2)
                                                text airline_sentiment
0                @VirginAmerica What @dhepburn said.           neutral
1  @VirginAmerica plus you've added commercials t...          positive
2  @VirginAmerica I didn't today... Must mean I n...           neutral
3  @VirginAmerica it's really aggressive to blast...          negative
4  @VirginAmerica and it's a really big bad thing...          negative


<ipython-input-7-5150b9e011b4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_df['cleaned_text'] = tweet_df['text'].apply(preprocess_text)
<ipython-input-7-5150b9e011b4>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_df['sentiment_label'] = tweet_df['airline_sentiment'].map({


In [8]:
tweet_df.head(5)

,text,airline_sentiment,cleaned_text,sentiment_label
0,@VirginAmerica What @dhepburn said.,neutral,virginamerica what dhepburn said,NaN
1,@VirginAmerica plus you've added commercials t...,positive,virginamerica plus youve added commercials to ...,1.0
2,@VirginAmerica I didn't today... Must mean I n...,neutral,virginamerica i didnt today must mean i need t...,NaN
3,@VirginAmerica it's really aggressive to blast...,negative,virginamerica its really aggressive to blast o...,0.0
4,@VirginAmerica and it's a really big bad thing...,negative,virginamerica and its a really big bad thing a...,0.0


In [9]:
# Filter out neutral sentiments (NaN values in 'sentiment_label')
tweet_df = tweet_df.dropna(subset=['sentiment_label'])

# Convert 'sentiment_label' to integers for training
tweet_df['sentiment_label'] = tweet_df['sentiment_label'].astype(int)

# Check the resulting dataset
print(tweet_df['sentiment_label'].value_counts())
print(tweet_df.shape)

sentiment_label
0    9178
1    2363
Name: count, dtype: int64
(11541, 4)


<ipython-input-9-cb649b7e1922>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweet_df['sentiment_label'] = tweet_df['sentiment_label'].astype(int)


In [10]:
tweet_df.head(5)

,text,airline_sentiment,cleaned_text,sentiment_label
1,@VirginAmerica plus you've added commercials t...,positive,virginamerica plus youve added commercials to ...,1
3,@VirginAmerica it's really aggressive to blast...,negative,virginamerica its really aggressive to blast o...,0
4,@VirginAmerica and it's a really big bad thing...,negative,virginamerica and its a really big bad thing a...,0
5,@VirginAmerica seriously would pay $30 a fligh...,negative,virginamerica seriously would pay a flight fo...,0
6,"@VirginAmerica yes, nearly every time I fly VX...",positive,virginamerica yes nearly every time i fly vx t...,1


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Use TF-IDF to convert text into numerical features
vectorizer = TfidfVectorizer(max_features=4)  # Use 4 features for 4 qubits
features = vectorizer.fit_transform(tweet_df['cleaned_text']).toarray()

# Normalize features for quantum compatibility
features = np.array([f / np.linalg.norm(f) if np.linalg.norm(f) > 0 else f for f in features])

# Get the labels
labels = tweet_df['sentiment_label'].values

In [13]:
n_qubits = 4  # Example: Replace with the actual number of features

# Define the quantum device
dev = qml.device("default.qubit", wires=n_qubits)

In [14]:
@qml.qnode(dev)
def quantum_circuit(feature_vector):
    # Encode features into quantum states
    for i, feature in enumerate(feature_vector):
        qml.RY(feature, wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Quantum feature extraction
def quantum_features(data):
    return np.array([quantum_circuit(f) for f in data])


In [16]:
# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Extract quantum features
X_train_quantum = quantum_features(X_train)
X_test_quantum = quantum_features(X_test)

In [17]:
# Train an SVM classifier
clf = SVC(kernel="linear")
clf.fit(X_train_quantum, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_quantum)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 80.64%


In [ ]:
# Define a simple quantum circuit for visualization
@qml.qnode(dev)
def quantum_circuit(feature_vector):
    # Encode features into quantum states
    for i, feature in enumerate(feature_vector):
        qml.RY(feature, wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Example feature vector
example_feature = [0.1, 0.5, -0.4, 0.2]  # Replace with actual normalized features

# Call the quantum circuit to visualize its structure
print(qml.draw(quantum_circuit)(example_feature))
